In [2]:
import pandas as pd
import numpy as np
from sodapy import Socrata
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

/Users/ishaanpoojari/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/ishaanpoojari/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [34]:
# Initialize the Socrata client and fetch data
client = Socrata("data.ny.gov", None)
results = client.get("wujg-7c2s", limit=1000000)
results


[{'transit_timestamp': '2023-01-16T10:00:00.000',
  'transit_mode': 'subway',
  'station_complex_id': '327',
  'station_complex': 'Chambers St (1,2,3)',
  'borough': 'Manhattan',
  'payment_method': 'metrocard',
  'fare_class_category': 'Metrocard - Seniors & Disability',
  'ridership': '10.0',
  'transfers': '4.0',
  'latitude': '40.715476989746094',
  'longitude': '-74.00926971435547',
  'georeference': {'type': 'Point',
   'coordinates': [-74.00926971435547, 40.715476989746094]},
  ':@computed_region_kjdx_g34t': '2095',
  ':@computed_region_yamh_8v7k': '749',
  ':@computed_region_wbg7_3whc': '723'},
 {'transit_timestamp': '2023-02-23T03:00:00.000',
  'transit_mode': 'subway',
  'station_complex_id': '327',
  'station_complex': 'Chambers St (1,2,3)',
  'borough': 'Manhattan',
  'payment_method': 'metrocard',
  'fare_class_category': 'Metrocard - Other',
  'ridership': '4.0',
  'transfers': '0.0',
  'latitude': '40.715476989746094',
  'longitude': '-74.00926971435547',
  'georeference

In [35]:

# Convert results to DataFrame
df = pd.DataFrame.from_records(results)

# Convert 'transit_timestamp' to datetime
df['transit_timestamp'] = pd.to_datetime(df['transit_timestamp'])

# Convert 'ridership' to numeric
df['ridership'] = pd.to_numeric(df['ridership'], errors='coerce')

print(df.head())
df

    transit_timestamp transit_mode station_complex_id      station_complex  \
0 2023-01-16 10:00:00       subway                327  Chambers St (1,2,3)   
1 2023-02-23 03:00:00       subway                327  Chambers St (1,2,3)   
2 2023-08-08 15:00:00       subway                327  Chambers St (1,2,3)   
3 2023-02-09 05:00:00       subway                327  Chambers St (1,2,3)   
4 2024-02-14 18:00:00       subway                 33            45 St (R)   

     borough payment_method               fare_class_category  ridership  \
0  Manhattan      metrocard  Metrocard - Seniors & Disability       10.0   
1  Manhattan      metrocard                 Metrocard - Other        4.0   
2  Manhattan      metrocard             Metrocard - Full Fare      188.0   
3  Manhattan      metrocard  Metrocard - Seniors & Disability        1.0   
4   Brooklyn      metrocard                 Metrocard - Other        5.0   

  transfers            latitude           longitude  \
0       4.0  40.715

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,georeference,:@computed_region_kjdx_g34t,:@computed_region_yamh_8v7k,:@computed_region_wbg7_3whc
0,2023-01-16 10:00:00,subway,327,"Chambers St (1,2,3)",Manhattan,metrocard,Metrocard - Seniors & Disability,10.0,4.0,40.715476989746094,-74.00926971435547,"{'type': 'Point', 'coordinates': [-74.00926971...",2095,749,723
1,2023-02-23 03:00:00,subway,327,"Chambers St (1,2,3)",Manhattan,metrocard,Metrocard - Other,4.0,0.0,40.715476989746094,-74.00926971435547,"{'type': 'Point', 'coordinates': [-74.00926971...",2095,749,723
2,2023-08-08 15:00:00,subway,327,"Chambers St (1,2,3)",Manhattan,metrocard,Metrocard - Full Fare,188.0,0.0,40.715476989746094,-74.00926971435547,"{'type': 'Point', 'coordinates': [-74.00926971...",2095,749,723
3,2023-02-09 05:00:00,subway,327,"Chambers St (1,2,3)",Manhattan,metrocard,Metrocard - Seniors & Disability,1.0,0.0,40.715476989746094,-74.00926971435547,"{'type': 'Point', 'coordinates': [-74.00926971...",2095,749,723
4,2024-02-14 18:00:00,subway,33,45 St (R),Brooklyn,metrocard,Metrocard - Other,5.0,0.0,40.648937,-74.01001,"{'type': 'Point', 'coordinates': [-74.01001, 4...",2090,894,899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,2023-01-10 19:00:00,subway,35,"59 St (N,R)",Brooklyn,omny,OMNY - Full Fare,65.0,1.0,40.641361236572266,-74.01788330078125,"{'type': 'Point', 'coordinates': [-74.01788330...",2090,894,899
999996,2023-09-07 06:00:00,subway,35,"59 St (N,R)",Brooklyn,omny,OMNY - Full Fare,203.0,5.0,40.641361236572266,-74.01788330078125,"{'type': 'Point', 'coordinates': [-74.01788330...",2090,894,899
999997,2023-06-06 21:00:00,subway,350,Pennsylvania Av (3),Brooklyn,metrocard,Metrocard - Fair Fare,6.0,0.0,40.664634704589844,-73.8948974609375,"{'type': 'Point', 'coordinates': [-73.89489746...",2090,894,885
999998,2023-01-01 19:00:00,subway,350,Pennsylvania Av (3),Brooklyn,metrocard,Metrocard - Full Fare,13.0,5.0,40.664634704589844,-73.8948974609375,"{'type': 'Point', 'coordinates': [-73.89489746...",2090,894,885


In [36]:
def get_average_ridership_by_station_and_time(dataframe, station_id, query_time):
    """
    Retrieve the average ridership for a given station and time, across the same day of the week.
    
    :param dataframe: DataFrame containing the MTA data
    :param station_id: ID or name of the station complex
    :param query_time: datetime or string representing the time to query (e.g., '08:00:00')
    :return: Average ridership for the given time and day of the week
    """
    # Ensure query_time is a datetime object for consistency
    if isinstance(query_time, str):
        query_time = pd.to_datetime(query_time)

    # Filter data for the given station
    station_filtered = dataframe[
        (dataframe['station_complex_id'] == station_id) |
        (dataframe['station_complex'].str.contains(station_id, case=False, na=False))
    ].copy()

    # Safely create new columns using loc
    station_filtered.loc[:, 'day_of_week'] = station_filtered['transit_timestamp'].dt.dayofweek
    station_filtered.loc[:, 'hour'] = station_filtered['transit_timestamp'].dt.hour

    # Filter by day of the week and hour
    day_hour_filtered = station_filtered[
        (station_filtered['day_of_week'] == query_time.dayofweek) &
        (station_filtered['hour'] == query_time.hour)
    ]

    # Calculate the average ridership
    average_ridership = day_hour_filtered['ridership'].mean()

    return average_ridership



In [37]:
# Example usage
station_complex_id = '283'  # Example Station ID or part of the name
query_time = '2021-01-01 08:00:00'  # Example query time
average_ridership = get_average_ridership_by_station_and_time(df, station_complex_id, query_time)
print(f"Average ridership for station {station_complex_id} on the same day of the week and time: {average_ridership}")

Average ridership for station 283 on the same day of the week and time: 38.0


In [38]:
earliest_timestamp = df['transit_timestamp'].min()
earliest_row = df[df['transit_timestamp'] == earliest_timestamp]

# Get the latest transit_timestamp
latest_timestamp = df['transit_timestamp'].max()
latest_row = df[df['transit_timestamp'] == latest_timestamp]

print("Earliest time:")
print(earliest_row["transit_timestamp"].unique()[0])
print("\nLatest time:")
print(latest_row["transit_timestamp"].unique()[0])

print("\nEarliest row(s):")
print(earliest_row)
print("\nLatest row(s):")
print(latest_row)

Earliest time:
2022-02-01 18:00:00

Latest time:
2024-04-07 00:00:00

Earliest row(s):
         transit_timestamp transit_mode station_complex_id  \
928773 2022-02-01 18:00:00       subway                 72   

               station_complex   borough payment_method fare_class_category  \
928773  Fort Hamilton Pkwy (N)  Brooklyn      metrocard   Metrocard - Other   

        ridership transfers            latitude          longitude  \
928773        3.0       0.0  40.631385803222656  -74.0053482055664   

                                             georeference  \
928773  {'type': 'Point', 'coordinates': [-74.00534820...   

       :@computed_region_kjdx_g34t :@computed_region_yamh_8v7k  \
928773                        2090                         894   

       :@computed_region_wbg7_3whc  
928773                         897  

Latest row(s):
       transit_timestamp transit_mode station_complex_id station_complex  \
720744        2024-04-07       subway                103    Bowery

In [39]:
# Playing around with data
import random

boroughs = df["borough"].unique()
print(boroughs)

# filtered_df = df[(df['borough'] == boroughs[random.randint(0, len(boroughs)-1)])]
borough_filtered = df[(df['borough'] == "Manhattan")]
borough_filtered.head(30)

['Manhattan' 'Brooklyn' 'Bronx' 'Queens' 'Staten Island']


,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,georeference,:@computed_region_kjdx_g34t,:@computed_region_yamh_8v7k,:@computed_region_wbg7_3whc
0,2023-01-16 10:00:00,subway,327,"Chambers St (1,2,3)",Manhattan,metrocard,Metrocard - Seniors & Disability,10.0,4.0,40.715476989746094,-74.00926971435547,"{'type': 'Point', 'coordinates': [-74.00926971...",2095,749,723
1,2023-02-23 03:00:00,subway,327,"Chambers St (1,2,3)",Manhattan,metrocard,Metrocard - Other,4.0,0.0,40.715476989746094,-74.00926971435547,"{'type': 'Point', 'coordinates': [-74.00926971...",2095,749,723
2,2023-08-08 15:00:00,subway,327,"Chambers St (1,2,3)",Manhattan,metrocard,Metrocard - Full Fare,188.0,0.0,40.715476989746094,-74.00926971435547,"{'type': 'Point', 'coordinates': [-74.00926971...",2095,749,723
3,2023-02-09 05:00:00,subway,327,"Chambers St (1,2,3)",Manhattan,metrocard,Metrocard - Seniors & Disability,1.0,0.0,40.715476989746094,-74.00926971435547,"{'type': 'Point', 'coordinates': [-74.00926971...",2095,749,723
6,2023-01-21 22:00:00,subway,327,"Chambers St (1,2,3)",Manhattan,metrocard,Metrocard - Seniors & Disability,4.0,0.0,40.715476989746094,-74.00926971435547,"{'type': 'Point', 'coordinates': [-74.00926971...",2095,749,723
7,2023-08-08 18:00:00,subway,327,"Chambers St (1,2,3)",Manhattan,metrocard,Metrocard - Other,83.0,0.0,40.715476989746094,-74.00926971435547,"{'type': 'Point', 'coordinates': [-74.00926971...",2095,749,723
8,2023-03-20 10:00:00,subway,327,"Chambers St (1,2,3)",Manhattan,metrocard,Metrocard - Seniors & Disability,10.0,0.0,40.715476989746094,-74.00926971435547,"{'type': 'Point', 'coordinates': [-74.00926971...",2095,749,723
9,2023-08-24 10:00:00,subway,327,"Chambers St (1,2,3)",Manhattan,metrocard,Metrocard - Full Fare,41.0,0.0,40.715476989746094,-74.00926971435547,"{'type': 'Point', 'coordinates': [-74.00926971...",2095,749,723
10,2023-07-25 23:00:00,subway,327,"Chambers St (1,2,3)",Manhattan,metrocard,Metrocard - Seniors & Disability,4.0,0.0,40.715476989746094,-74.00926971435547,"{'type': 'Point', 'coordinates': [-74.00926971...",2095,749,723
11,2023-02-27 06:00:00,subway,327,"Chambers St (1,2,3)",Manhattan,metrocard,Metrocard - Other,4.0,0.0,40.715476989746094,-74.00926971435547,"{'type': 'Point', 'coordinates': [-74.00926971...",2095,749,723


In [40]:
station_filted = df[(df['station_complex'] == "7 Av (E,B,D)")]
station_filted

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,georeference,:@computed_region_kjdx_g34t,:@computed_region_yamh_8v7k,:@computed_region_wbg7_3whc
1003,2024-01-26 00:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Unlimited 30-Day,25.0,0.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742
1092,2024-01-28 01:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Unlimited 7-Day,13.0,0.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742
1864,2024-01-27 11:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Other,5.0,0.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742
1876,2024-01-28 00:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Unlimited 30-Day,21.0,0.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742
2657,2024-01-25 17:00:00,subway,277,"7 Av (E,B,D)",Manhattan,omny,OMNY - Full Fare,1424.0,3.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983330,2024-01-25 12:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Other,20.0,0.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742
987344,2024-01-28 20:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Unlimited 30-Day,27.0,0.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742
989083,2024-01-28 05:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Fair Fare,1.0,0.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742
995586,2024-01-27 08:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Other,15.0,0.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742


In [41]:
# Preprocessing
aggregations = {
    'transit_timestamp': 'first',
    'ridership': 'sum',  # Sum the ridership for each group
    'transfers': 'sum',  # Also summing the transfers if needed
    # For all other columns that you want to keep the same and which do not have multiple different values per group, you can use 'first' or 'max'
    
    'transit_mode': 'first',
    'borough': 'first',
    'latitude': 'first',
    'longitude': 'first',
    'georeference': 'first'
}

# Perform the groupby and aggregate
condensed_df = df.groupby(['station_complex_id', 'station_complex', 'transit_timestamp'], as_index=False).agg(aggregations)
condensed_df

,station_complex_id,station_complex,transit_timestamp,ridership,transfers,transit_mode,borough,latitude,longitude,georeference
0,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 03:00:00,1.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4..."
1,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 04:00:00,5.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4..."
2,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 07:00:00,72.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4..."
3,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 08:00:00,290.0,0.00.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4..."
4,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 10:00:00,59.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4..."
...,...,...,...,...,...,...,...,...,...,...
234863,TRAM2,RI Tramway (Roosevelt),2024-03-07 20:00:00,2.0,0.0,tram,Manhattan,40.75734,-73.95412,"{'type': 'Point', 'coordinates': [-73.95412, 4..."
234864,TRAM2,RI Tramway (Roosevelt),2024-03-09 12:00:00,1.0,0.0,tram,Manhattan,40.75734,-73.95412,"{'type': 'Point', 'coordinates': [-73.95412, 4..."
234865,TRAM2,RI Tramway (Roosevelt),2024-03-15 19:00:00,6.0,0.0,tram,Manhattan,40.75734,-73.95412,"{'type': 'Point', 'coordinates': [-73.95412, 4..."
234866,TRAM2,RI Tramway (Roosevelt),2024-03-19 17:00:00,1.0,0.0,tram,Manhattan,40.75734,-73.95412,"{'type': 'Point', 'coordinates': [-73.95412, 4..."


In [42]:
# Filtering for subway data
subway_df = condensed_df[condensed_df['transit_mode'] == 'subway']
subway_df

,station_complex_id,station_complex,transit_timestamp,ridership,transfers,transit_mode,borough,latitude,longitude,georeference
0,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 03:00:00,1.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4..."
1,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 04:00:00,5.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4..."
2,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 07:00:00,72.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4..."
3,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 08:00:00,290.0,0.00.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4..."
4,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 10:00:00,59.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4..."
...,...,...,...,...,...,...,...,...,...,...
234607,99,"Lorimer St (M,J)",2024-03-07 07:00:00,11.0,0.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4..."
234608,99,"Lorimer St (M,J)",2024-03-07 08:00:00,25.0,0.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4..."
234609,99,"Lorimer St (M,J)",2024-03-07 19:00:00,9.0,0.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4..."
234610,99,"Lorimer St (M,J)",2024-03-07 20:00:00,1.0,0.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4..."


In [43]:

final_df = subway_df.dropna(subset=['ridership'])
final_df['month'] = final_df['transit_timestamp'].dt.month
final_df['day_of_week'] = final_df['transit_timestamp'].dt.dayofweek
final_df['hour'] = final_df['transit_timestamp'].dt.hour
final_df

,station_complex_id,station_complex,transit_timestamp,ridership,transfers,transit_mode,borough,latitude,longitude,georeference,month,day_of_week,hour
0,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 03:00:00,1.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",1,3,3
1,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 04:00:00,5.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",1,3,4
2,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 07:00:00,72.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",1,3,7
3,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 08:00:00,290.0,0.00.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",1,3,8
4,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 10:00:00,59.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",1,3,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
234607,99,"Lorimer St (M,J)",2024-03-07 07:00:00,11.0,0.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4...",3,3,7
234608,99,"Lorimer St (M,J)",2024-03-07 08:00:00,25.0,0.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4...",3,3,8
234609,99,"Lorimer St (M,J)",2024-03-07 19:00:00,9.0,0.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4...",3,3,19
234610,99,"Lorimer St (M,J)",2024-03-07 20:00:00,1.0,0.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4...",3,3,20


In [44]:
# Extracting relavent features and target variable
X = final_df[['hour', 'day_of_week', 'month']]
y = final_df['ridership']

# Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
# Model Training
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [46]:
predictions = model.predict(X_test)

In [47]:
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(f"Root Mean Square Error: {rmse}")

Root Mean Square Error: 228.80961453548636


In [48]:
# Predictions
def predict_ridership(hour, day_of_week, month):
    input_data = np.array([[hour, day_of_week, month]])
    predicted_ridership = model.predict(input_data)
    return predicted_ridership[0]

hour = 8  # 8 AM
day_of_week = 4  # Friday
month = 1  # January
predicted_ridership = predict_ridership(hour, day_of_week, month)
print(f"Predicted ridership: {predicted_ridership}")

Predicted ridership: 229.31553682788086


/Users/ishaanpoojari/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [49]:
fridays_in_january_at_8am = final_df[(final_df['day_of_week'] == 4) & (final_df['month'] == 1) & (final_df['hour'] == 8)]
fridays_in_january_at_8am

,station_complex_id,station_complex,transit_timestamp,ridership,transfers,transit_mode,borough,latitude,longitude,georeference,month,day_of_week,hour
37,1,"Astoria-Ditmars Blvd (N,W)",2024-01-26 08:00:00,4.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",1,4,8
191,10,"49 St (N,R,W)",2024-01-26 08:00:00,239.0,0.00.00.0,subway,Manhattan,40.7599,-73.98414,"{'type': 'Point', 'coordinates': [-73.98414, 4...",1,4,8
333,100,"Hewes St (M,J)",2024-01-19 08:00:00,94.0,0.0,subway,Brooklyn,40.70687,-73.95343,"{'type': 'Point', 'coordinates': [-73.95343, 4...",1,4,8
361,100,"Hewes St (M,J)",2024-01-26 08:00:00,17.0,0.0,subway,Brooklyn,40.70687,-73.95343,"{'type': 'Point', 'coordinates': [-73.95343, 4...",1,4,8
523,101,"Marcy Av (M,J,Z)",2024-01-26 08:00:00,432.0,14.0,subway,Brooklyn,40.70836,-73.957756,"{'type': 'Point', 'coordinates': [-73.957756, ...",1,4,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
233886,95,"Gates Av (J,Z)",2024-01-26 08:00:00,96.0,0.04.0,subway,Brooklyn,40.68963,-73.92227,"{'type': 'Point', 'coordinates': [-73.92227, 4...",1,4,8
234038,96,Kosciuszko St (J),2024-01-26 08:00:00,49.0,7.0,subway,Brooklyn,40.69334,-73.92882,"{'type': 'Point', 'coordinates': [-73.92882, 4...",1,4,8
234192,97,"Myrtle Av (M,J,Z)",2024-01-26 08:00:00,105.0,0.0,subway,Brooklyn,40.69721,-73.93565,"{'type': 'Point', 'coordinates': [-73.93565, 4...",1,4,8
234348,98,"Flushing Av (M,J)",2024-01-26 08:00:00,133.0,6.0,subway,Brooklyn,40.70026,-73.941124,"{'type': 'Point', 'coordinates': [-73.941124, ...",1,4,8


In [50]:
final_df.to_json("HistoricalRidership.json", orient='records', lines=True)

print("DataFrame has been successfully saved as JSON.")

DataFrame has been successfully saved as JSON.


In [51]:
final_df2 = subway_df.dropna(subset=['ridership'])
final_df2['station_complex_id'] = final_df2['station_complex_id'].astype(int)

# Feature Engineering
final_df2['hour'] = final_df2['transit_timestamp'].dt.hour
final_df2['day_of_week'] = final_df2['transit_timestamp'].dt.dayofweek
final_df2


,station_complex_id,station_complex,transit_timestamp,ridership,transfers,transit_mode,borough,latitude,longitude,georeference,hour,day_of_week
0,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 03:00:00,1.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",3,3
1,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 04:00:00,5.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",4,3
2,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 07:00:00,72.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",7,3
3,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 08:00:00,290.0,0.00.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",8,3
4,1,"Astoria-Ditmars Blvd (N,W)",2024-01-11 10:00:00,59.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",10,3
...,...,...,...,...,...,...,...,...,...,...,...,...
234607,99,"Lorimer St (M,J)",2024-03-07 07:00:00,11.0,0.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4...",7,3
234608,99,"Lorimer St (M,J)",2024-03-07 08:00:00,25.0,0.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4...",8,3
234609,99,"Lorimer St (M,J)",2024-03-07 19:00:00,9.0,0.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4...",19,3
234610,99,"Lorimer St (M,J)",2024-03-07 20:00:00,1.0,0.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4...",20,3


In [52]:
# Select relevant features for the model, now excluding 'month'
X = final_df2[['station_complex_id', 'day_of_week', 'hour']]
y = final_df2['ridership']

In [53]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
predictions = model.predict(X_test)

In [96]:
# Evaluate the model
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(f"Root Mean Square Error: {rmse}")

def predict_ridership(station_complex_id, day_of_week, hour):
    # Format the input features as a DataFrame
    input_features = pd.DataFrame({
        'station_complex_id': [station_complex_id],
        'day_of_week': [day_of_week],
        'hour': [hour]
    })
    # Predict ridership
    predicted_ridership = model.predict(input_features)
    return predicted_ridership[0]

# Example usage
station_id_example = 283  # Replace with an actual station_complex_id
day_of_week_example = 6  # Friday
hour_example = 3 # 8 AM
prediction = predict_ridership(station_id_example, day_of_week_example, hour_example)
print(f"Predicted ridership for station ID {station_id_example} on day {day_of_week_example} at hour {hour_example}: {prediction}")

Root Mean Square Error: 151.42993286541386
Predicted ridership for station ID 283 on day 6 at hour 3: 2.722


In [94]:
actual_data = final_df2[(final_df2['station_complex_id'] == 167) & (final_df2['day_of_week'] == 6) & (final_df2['hour'] == 4)]
actual_data['ridership'].mean()

69.0

In [89]:
final_df2[final_df2["station_complex_id"] == 167]

,station_complex_id,station_complex,transit_timestamp,ridership,transfers,transit_mode,borough,latitude,longitude,georeference,hour,day_of_week
9260,167,"W 4 St-Wash Sq (A,C,E,B,D,F,M)",2023-12-19 00:00:00,78.0,0.0,subway,Manhattan,40.732338,-74.000496,"{'type': 'Point', 'coordinates': [-74.000496, ...",0,1
9261,167,"W 4 St-Wash Sq (A,C,E,B,D,F,M)",2024-01-11 01:00:00,14.0,0.0,subway,Manhattan,40.732338,-74.000496,"{'type': 'Point', 'coordinates': [-74.000496, ...",1,3
9262,167,"W 4 St-Wash Sq (A,C,E,B,D,F,M)",2024-01-11 03:00:00,20.0,0.0,subway,Manhattan,40.732338,-74.000496,"{'type': 'Point', 'coordinates': [-74.000496, ...",3,3
9263,167,"W 4 St-Wash Sq (A,C,E,B,D,F,M)",2024-01-11 08:00:00,1389.0,12.0,subway,Manhattan,40.732338,-74.000496,"{'type': 'Point', 'coordinates': [-74.000496, ...",8,3
9264,167,"W 4 St-Wash Sq (A,C,E,B,D,F,M)",2024-01-11 12:00:00,171.0,0.01.0,subway,Manhattan,40.732338,-74.000496,"{'type': 'Point', 'coordinates': [-74.000496, ...",12,3
...,...,...,...,...,...,...,...,...,...,...,...,...
9404,167,"W 4 St-Wash Sq (A,C,E,B,D,F,M)",2024-03-07 14:00:00,67.0,0.0,subway,Manhattan,40.732338,-74.000496,"{'type': 'Point', 'coordinates': [-74.000496, ...",14,3
9405,167,"W 4 St-Wash Sq (A,C,E,B,D,F,M)",2024-03-07 15:00:00,225.0,0.0,subway,Manhattan,40.732338,-74.000496,"{'type': 'Point', 'coordinates': [-74.000496, ...",15,3
9406,167,"W 4 St-Wash Sq (A,C,E,B,D,F,M)",2024-03-07 16:00:00,1488.0,5.0,subway,Manhattan,40.732338,-74.000496,"{'type': 'Point', 'coordinates': [-74.000496, ...",16,3
9407,167,"W 4 St-Wash Sq (A,C,E,B,D,F,M)",2024-03-09 06:00:00,3.0,0.0,subway,Manhattan,40.732338,-74.000496,"{'type': 'Point', 'coordinates': [-74.000496, ...",6,5


In [57]:
errors = abs(predictions - y_test)
mape = 100 * (errors / y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')
mape

Accuracy: -87.21 %.


234358    406.303922
111853     37.188937
86502      69.036452
160113     12.247889
156708     15.773029
             ...    
206363    748.800000
45709      25.174784
52145       5.536528
21452      73.548611
90742       2.528701
Name: ridership, Length: 46874, dtype: float64

In [58]:
# Stations with the most data - NOT the most ridership
station_counts = final_df2['station_complex_id'].value_counts().reset_index()
station_names = final_df2[['station_complex_id', 'station_complex']].drop_duplicates()
station_counts_with_names = station_counts.merge(station_names, on='station_complex_id', how='left')

print(station_counts_with_names.head(25))


    station_complex_id  count                     station_complex
0                   34   8352                           53 St (R)
1                   33   8341                           45 St (R)
2                  345   8341            Crown Hts-Utica Av (3,4)
3                  344   8324                     Kingston Av (3)
4                  337   8320                 Nevins St (2,3,4,5)
5                  343   8318                     Nostrand Av (3)
6                  346   8307            Sutter Av-Rutland Rd (3)
7                  347   8291                     Saratoga Av (3)
8                  328   8287                   WTC Cortlandt (1)
9                  327   8284                 Chambers St (1,2,3)
10                 348   8269                     Rockaway Av (3)
11                 349   8257                       Junius St (3)
12                 340   8210              Grand Army Plaza (2,3)
13                 329   8199                       Rector St (1)
14        

In [59]:
top_10_station_ids = station_counts_with_names['station_complex_id'].head(10).tolist()
top_10_station_ids

[34, 33, 345, 344, 337, 343, 346, 347, 328, 327]

In [60]:
# Repeated trials testing

def test_ridership_predictions(station_ids, days_of_week, hours):
    results = []

    for station_id in station_ids:
        for day in days_of_week:
            for hour in hours:
                # Predict ridership
                prediction = predict_ridership(station_id, day, hour)

                # Extract actual data
                actual_data = final_df2[
                    (final_df2['station_complex_id'] == station_id) &
                    (final_df2['day_of_week'] == day) &
                    (final_df2['hour'] == hour)
                ]

                # Compute actual mean and median ridership
                actual_mean = actual_data['ridership'].mean()
                actual_median = actual_data['ridership'].median()

                # Calculate the percentage error between predicted and actual mean
                percentage_error_mean = 100 * abs(prediction - actual_mean) / actual_mean if actual_mean > 0 else None
                # Calculate the percentage error between predicted and actual median
                percentage_error_median = 100 * abs(prediction - actual_median) / actual_median if actual_median > 0 else None

                # Record the results
                results.append({
                    'Station ID': station_id,
                    'Day of Week': day,
                    'Hour': hour,
                    'Predicted Ridership': prediction,
                    'Actual Mean Ridership': actual_mean,
                    'Actual Median Ridership': actual_median,
                    'Percentage Error (Mean)': percentage_error_mean,
                    'Percentage Error (Median)': percentage_error_median
                })

                # Print the results for each prediction
                print(f"Station ID {station_id}, Day {day}, Hour {hour}")
                print(f"Predicted Ridership: {prediction}")
                print(f"Actual Mean Ridership: {actual_mean}")
                print(f"Actual Median Ridership: {actual_median}")
                print(f"Percentage Error (Mean): {percentage_error_mean}%")
                print(f"Percentage Error (Median): {percentage_error_median}%")
                print('-------------------------------------')

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)

    # Compute overall percentage error
    overall_percentage_error_mean = results_df['Percentage Error (Mean)'].mean()
    overall_percentage_error_median = results_df['Percentage Error (Median)'].mean()

    print(f"Overall Percentage Error (Mean): {overall_percentage_error_mean}%")
    print(f"Overall Percentage Error (Median): {overall_percentage_error_median}%")

    return results_df

# Example usage with a list of station IDs, days, and hours
station_ids = top_10_station_ids  # Replace with your actual list of station IDs
days_of_week = [0, 1, 2, 3, 4, 5, 6]  # 0 = Monday, 6 = Sunday
hours = [8, 14, 20]  # Example set of hours in 24-hour format

# Call the test function
test_results_df = test_ridership_predictions(station_ids, days_of_week, hours)


Station ID 34, Day 0, Hour 8
Predicted Ridership: 550.8724297068394
Actual Mean Ridership: 561.9387755102041
Actual Median Ridership: 605.0
Percentage Error (Mean): 1.9693152146899224%
Percentage Error (Median): 8.946705833580266%
-------------------------------------


Station ID 34, Day 0, Hour 14
Predicted Ridership: 244.95185978440688
Actual Mean Ridership: 246.12244897959184
Actual Median Ridership: 293.0
Percentage Error (Mean): 0.4756125254068243%
Percentage Error (Median): 16.398682667437924%
-------------------------------------
Station ID 34, Day 0, Hour 20
Predicted Ridership: 108.33297912403474
Actual Mean Ridership: 107.44897959183673
Actual Median Ridership: 122.0
Percentage Error (Mean): 0.8227156140114378%
Percentage Error (Median): 11.202476127840379%
-------------------------------------
Station ID 34, Day 1, Hour 8
Predicted Ridership: 675.9106452033915
Actual Mean Ridership: 671.4081632653061
Actual Median Ridership: 708.0
Percentage Error (Mean): 0.6706027993743932%
Percentage Error (Median): 4.532394745283689%
-------------------------------------
Station ID 34, Day 1, Hour 14
Predicted Ridership: 231.71254630373548
Actual Mean Ridership: 237.32
Actual Median Ridership: 271.5
Percentage Error (Mean): 2.3628239070725243%
Percentag

In [61]:

def test_ridership_predictions(station_ids, days_of_week, hours):
    results = []

    for station_id in station_ids:
        for day in days_of_week:
            for hour in hours:
                # Predict ridership
                prediction = predict_ridership(station_id, day, hour)

                # Extract actual data
                actual_data = final_df2[
                    (final_df2['station_complex_id'] == station_id) &
                    (final_df2['day_of_week'] == day) &
                    (final_df2['hour'] == hour)
                ]

                # Compute actual mean and median ridership
                actual_mean = actual_data['ridership'].mean()
                actual_median = actual_data['ridership'].median()

                # Calculate the percentage error between predicted and actual mean
                percentage_error_mean = 100 * abs(prediction - actual_mean) / actual_mean if actual_mean > 0 else None
                # Calculate the percentage error between predicted and actual median
                percentage_error_median = 100 * abs(prediction - actual_median) / actual_median if actual_median > 0 else None

                # Record the results
                results.append({
                    'Station ID': station_id,
                    'Day of Week': day,
                    'Hour': hour,
                    'Predicted Ridership': prediction,
                    'Actual Mean Ridership': actual_mean,
                    'Actual Median Ridership': actual_median,
                    'Percentage Error (Mean)': percentage_error_mean,
                    'Percentage Error (Median)': percentage_error_median
                })

                # Print the results for each prediction
                print(f"Station ID {station_id}, Day {day}, Hour {hour}")
                print(f"Predicted Ridership: {prediction}")
                print(f"Actual Mean Ridership: {actual_mean}")
                print(f"Actual Median Ridership: {actual_median}")
                print(f"Percentage Error (Mean): {percentage_error_mean}%")
                print(f"Percentage Error (Median): {percentage_error_median}%")
                print('-------------------------------------')

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)

    # Compute overall percentage error
    overall_percentage_error_mean = results_df['Percentage Error (Mean)'].mean()
    overall_percentage_error_median = results_df['Percentage Error (Median)'].mean()

    print(f"Overall Percentage Error (Mean): {overall_percentage_error_mean}%")
    print(f"Overall Percentage Error (Median): {overall_percentage_error_median}%")

    return results_df

# Example usage with a list of station IDs, days, and hours
station_ids = top_10_station_ids  # Replace with your actual list of station IDs
days_of_week = [0, 1, 2, 3, 4, 5, 6]  # 0 = Monday, 6 = Sunday
hours = [8, 14, 20]  # Example set of hours in 24-hour format

# Call the test function
test_results_df = test_ridership_predictions(station_ids, days_of_week, hours)

Station ID 34, Day 0, Hour 8
Predicted Ridership: 550.8724297068394
Actual Mean Ridership: 561.9387755102041
Actual Median Ridership: 605.0
Percentage Error (Mean): 1.9693152146899224%
Percentage Error (Median): 8.946705833580266%
-------------------------------------
Station ID 34, Day 0, Hour 14
Predicted Ridership: 244.95185978440688
Actual Mean Ridership: 246.12244897959184
Actual Median Ridership: 293.0
Percentage Error (Mean): 0.4756125254068243%
Percentage Error (Median): 16.398682667437924%
-------------------------------------
Station ID 34, Day 0, Hour 20
Predicted Ridership: 108.33297912403474
Actual Mean Ridership: 107.44897959183673
Actual Median Ridership: 122.0
Percentage Error (Mean): 0.8227156140114378%
Percentage Error (Median): 11.202476127840379%
-------------------------------------
Station ID 34, Day 1, Hour 8
Predicted Ridership: 675.9106452033915
Actual Mean Ridership: 671.4081632653061
Actual Median Ridership: 708.0
Percentage Error (Mean): 0.6706027993743932%


In [66]:

# Sample DataFrame creation (Replace this with your actual DataFrame)
# Assuming 'df' is your DataFrame with columns 'station_complex_id', 'ridership'
data = {
    'station_complex_id': ['Station1', 'Station1', 'Station2', 'Station2', 'Station3'],
    'ridership': [100, 200, 150, 250, 300]
}
df = pd.DataFrame(data)

# Group by 'station_complex_id' and calculate the mean ridership
average_ridership = df.groupby('station_complex_id')['ridership'].mean()

# Sort the average ridership in descending order
sorted_average_ridership = average_ridership.sort_values(ascending=False).reset_index()

# Rename columns for clarity
sorted_average_ridership.columns = ['Station', 'Average Ridership']

# Print the sorted DataFrame
print(sorted_average_ridership)


    Station  Average Ridership
0  Station3              300.0
1  Station2              200.0
2  Station1              150.0


In [73]:
average_ridership_per_station = final_df2.groupby(['station_complex_id', 'station_complex'])['ridership'].mean().reset_index()
sorted_stations_by_ridership = average_ridership_per_station.sort_values(by='ridership', ascending=False)
sorted_stations_by_ridership
top_10_station_ids = sorted_stations_by_ridership['station_complex_id'].head(10).tolist()

# Print the list of the top 10 station IDs
print(top_10_station_ids)
sorted_stations_by_ridership.head(25)

[611, 610, 607, 602, 164, 614, 628, 616, 345, 318]


,station_complex_id,station_complex,ridership
402,611,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",997.112500
401,610,"Grand Central-42 St (S,4,5,6,7)",814.157233
398,607,"34 St-Herald Sq (B,D,F,M,N,Q,R,W)",684.139394
393,602,"14 St-Union Sq (L,N,Q,R,W,4,5,6)",618.845161
130,164,"34 St-Penn Station (A,C,E)",550.701220
405,614,"59 St-Columbus Circle (A,B,C,D,1)",528.875000
419,628,"Fulton St (A,C,J,Z,2,3,4,5)",522.347561
407,616,"74-Broadway (7)/Jackson Hts-Roosevelt Av (E,F,...",490.846667
282,345,"Crown Hts-Utica Av (3,4)",487.552452
262,318,"34 St-Penn Station (1,2,3)",468.907285


In [95]:
from joblib import dump

# Save the model as a file named 'random_forest_model.joblib' in your current directory
dump(model, 'models/millionDataModel.joblib')

['millionDataModel.joblib']